In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Dot, Activation, Concatenate
import numpy as np


In [2]:
english_sentences = [
    "she eats bread",
    "he drinks water",
    "we play football",
    "they read books",
    "he writes a letter"
]

In [3]:
tamil_sentences = [
    " அவள் ரொட்டி சாப்பிடுகிறாள் ",
    " அவன் தண்ணீர் குடிக்கிறான் ",
    " நாங்கள் கால்பந்து விளையாடுகிறோம் ",
    " அவர்கள் புத்தகங்கள் படிக்கிறார்கள் ",
    " அவன் ஒரு கடிதம் எழுதுகிறான் "
]

In [4]:
src_tokenizer = Tokenizer(oov_token="")
src_tokenizer.fit_on_texts(english_sentences)
src_seq = src_tokenizer.texts_to_sequences(english_sentences)

tgt_tokenizer = Tokenizer(filters='', oov_token="")
tgt_tokenizer.fit_on_texts(tamil_sentences)
tgt_seq = tgt_tokenizer.texts_to_sequences(tamil_sentences)

src_vocab_size = len(src_tokenizer.word_index) + 1
tgt_vocab_size = len(tgt_tokenizer.word_index) + 1

In [5]:
max_src_len = max(len(s) for s in src_seq)
max_tgt_len = max(len(s) for s in tgt_seq)
encoder_input = pad_sequences(src_seq, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(
    [s[:-1] for s in tgt_seq],
    maxlen=max_tgt_len-1,
    padding='post'
)
decoder_target = pad_sequences(
    [s[1:] for s in tgt_seq],
    maxlen=max_tgt_len-1,
    padding='post'
)

In [6]:
embedding_dim = 32
latent_dim = 64

In [7]:
encoder_inputs = Input(shape=(max_src_len,))
enc_emb = Embedding(src_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [8]:
decoder_inputs = Input(shape=(max_tgt_len-1,))
dec_emb = Embedding(tgt_vocab_size, embedding_dim)(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)

In [9]:
attention = Dot(axes=[2, 2])([decoder_outputs, encoder_outputs])
attention = Activation('softmax')(attention)

context = Dot(axes=[2, 1])([attention, encoder_outputs])
decoder_combined = Concatenate(axis=-1)([context, decoder_outputs])

In [10]:
output = Dense(tgt_vocab_size, activation='softmax')(decoder_combined)

In [11]:
model = Model([encoder_inputs, decoder_inputs], output)
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 4)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 3)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 4, 32)     │        544 │ input_layer[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 3, 32)     │        544 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ [(None, 4, 64),   │     24,832 │ embedding[0][0]   │
│                     │ (None, 64),       │            │                   │
│                     │ (None, 64)]       │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ [(None, 3, 64),   │     24,832 │ embedding_1[0][0… │
│                     │ (None, 64),       │            │ lstm[0][1],       │
│                     │ (None, 64)]       │            │ lstm[0][2]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot (Dot)           │ (None, 3, 4)      │          0 │ lstm_1[0][0],     │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 3, 4)      │          0 │ dot[0][0]         │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dot_1 (Dot)         │ (None, 3, 64)     │          0 │ activation[0][0], │
│                     │                   │            │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 3, 128)    │          0 │ dot_1[0][0],      │
│ (Concatenate)       │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 3, 17)     │      2,193 │ concatenate[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 52,945 (206.82 KB)

 Trainable params: 52,945 (206.82 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
model.fit(
    [encoder_input, decoder_input],
    np.expand_dims(decoder_target, -1),
    batch_size=4,
    epochs=40,
    shuffle=False,
    verbose=1
)

Epoch 1/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0086
Epoch 2/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0085
Epoch 3/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0084
Epoch 4/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0083
Epoch 5/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0082
Epoch 6/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0082
Epoch 7/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0081
Epoch 8/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 1.0000 - loss: 0.0080
Epoch 9/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 1.0000 - loss: 0.0079
Epoch 10/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0078
Epoch 11/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 1.0000 - loss: 0.0078
Epoch 12/40
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 1.0000 - loss: 0.0077
E

In [25]:
def translate(sentence):
    seq = src_tokenizer.texts_to_sequences([sentence])
    seq = pad_sequences(seq, maxlen=max_src_len, padding='post')

    target_seq = np.zeros((1, max_tgt_len-1))
    target_seq[0, 0] = tgt_tokenizer.word_index['']

    decoded = []

    for i in range(max_tgt_len-1):
        preds = model.predict([seq, target_seq], verbose=0)
        word_id = np.argmax(preds[0, i])
        word = reverse_tgt_index.get(word_id, '')

        if word == '' or word == '':
            break

        decoded.append(word)
        target_seq[0, i+1] = word_id
    return ' '.join(decoded)

In [27]:
print("English :", "she eats bread")
print("Tamil   :", translate("she eats bread"))

print("English :", "he drinks water")
print("Tamil   :", translate("he drinks water"))

English : she eats bread
Tamil   : ரொட்டி சாப்பிடுகிறாள்
English : he drinks water
Tamil   : தண்ணீர் குடிக்கிறான்
